### Part A


#### Task 1


I recommend using an embedded data model where each document represents one day of climate data, with any associated hotspot events for that day embedded as an array within the climate document.

The first step in designing the schema was to identify relationships between the climate and hotspot datasets. There is a clear one-to-many relationship between them - each climate record for a given date can have zero, one, or many associated hotspot records. Embedding the hotspots within the corresponding climate document keeps this related data together and aligns the schema with the natural relationships.

The key benefit of this embedded design is that it matches the primary data access patterns. The application will frequently need to retrieve the climate data for a specific date along with any hotspots that occurred. By embedding hotspots, all related data can be fetched with a single query, eliminating costly joins and reducing read operations. This is much more efficient than constantly joining climate and hotspot data from separate collections. Additionally, embedding the hotspots provides atomic updates. When new hotspot data arrives for a date, it can be pushed into the hotspots array atomically as part of a single update operation to the climate document.

Embedding also simplifies the processing of streaming climate and hotspot data. As new hotspot events arrive, they can easily be appended to the hotspots array in the corresponding climate document for that day. The data is automatically structured in the optimal format for querying. A reference model would add complexity, needing to manage climate and hotspot data separately and link them.
Additionally, embedding enables efficient determination of the cause of each fire (natural vs other). The relevant climate conditions like temperature and GHI are in the same document as the hotspot, so a simple query can check the values and set the cause field appropriately before inserting the hotspot. A reference model would require extra queries to fetch the related climate data to evaluate the fire cause.

While embedding introduces some data redundancy, like repeating date information, the benefits outweigh this acceptable trade-off. The number of hotspots per day is likely small and bounded, so document sizes should stay below MongoDB's 16 MB limit. Duplication is minimal and, since climate and hotspot data are append-only, keeping duplicated data in sync is not a significant challenge.
Finally, the embedded schema aligns with the typical usage patterns. Climate is the core dataset that will often be queried alone, while hotspots are usually accessed in the context of their climate conditions. Embedding hotspots as child data within climate documents matches this contextual relationship.

In conclusion, I recommend an embedded data model for the StopFire project.

Example:

```
{
  "_id": ObjectId("663e3650c366387f1b054c70"),
  "date": ISODate("2023-12-13T00:00:00.000Z"),
  "station": "948702",
  "climate": {
    "air_temperature_celcius": 28,
    "relative_humidity": 49.9,
    "windspeed_knots": 11,
    "max_wind_speed": 19,
    "precipitation": "0.00I",
    "ghi_wm2": 240
  },
  "hotspots": [
    {
      "latitude": -36.6296,
      "longitude": 142.5191,
      "datetime": ISODate("2023-12-13T00:00:00.000Z"),
      "confidence": 70,
      "surface_temperature_celcius": 45
    },
    {
      "latitude": -37.6296,
      "longitude": 142.5191,
      "datetime": ISODate("2023-12-13T00:00:00.000Z"),
      "confidence": 75,
      "surface_temperature_celcius": 42
    },
    {
      "latitude": -38.6296,
      "longitude": 142.5191,
      "datetime": ISODate("2023-12-13T00:00:00.000Z"),
      "confidence": 72,
      "surface_temperature_celcius": 49
    }
  ]
}
```


#### Task 2


##### Task 2.1


In [ ]:
# import all the necessary libraries
from datetime import datetime, time
import pprint
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import json
import sys
import os

In [ ]:
# change directory (local machine only)
# os.chdir("A2")
# os.chdir("data")

# dirs = os.listdir(os.getcwd())
# print(dirs)

In [ ]:
hostip = "192.168.10.125"

In [ ]:
# connect to mongo
client = pymongo.MongoClient(hostip, 27017)

# drop db
client.drop_database("fit3182_assignment_db")

# create a new database
db = client["fit3182_assignment_db"]

# create a new collection
collection = db["climate"]

In [ ]:
# drop collection
collection.drop()

In [ ]:
# helper function to convert date to datetime
def date_to_datetime(date_obj):
    return datetime.combine(date_obj, time(0, 0))

In [ ]:
# read hotspot data and insert into mongo
hotspot_data = pd.read_csv("hotspot_historic.csv")

# to store hotspot data
hotspot_dict = {}

# iterate through hotspot data and store in dictionary
for _, row in hotspot_data.iterrows():

    # convert date to datetime using helper function
    date = date_to_datetime(datetime.strptime(row["date"], "%d/%m/%Y").date())

    # if date not in dictionary, add it
    if date not in hotspot_dict:
        hotspot_dict[date] = []

    # append hotspot data to dictionary
    hotspot_dict[date].append(
        {
            "latitude": row["latitude"],
            "longitude": row["longitude"],
            "datetime": datetime.strptime(
                row["datetime"], "%Y-%m-%d %H:%M:%S"
            ).strftime("%H:%M:%S"),
            "confidence": row["confidence"],
            "surface_temperature_celcius": row["surface_temperature_celcius"],
        }
    )

# read climate data and insert into mongo
climate_data = pd.read_csv("climate_historic.csv")

# iterate through climate data and insert into mongo, same as above
for _, row in climate_data.iterrows():
    date = date_to_datetime(datetime.strptime(row["date"], "%d/%m/%Y").date())

    # remove whitespace from precipitation
    precipitation = row["precipitation"].strip()

    # set climate document
    climate_doc = {
        "date": date.strftime("%Y-%m-%dT%H:%M:%S"),
        "station": int(row["station"]),
        "climate": {
            "air_temperature_celcius": int(row["air_temperature_celcius"]),
            "relative_humidity": float(row["relative_humidity"]),
            "windspeed_knots": float(row["windspeed_knots"]),
            "max_wind_speed": float(row["max_wind_speed"]),
            "precipitation": precipitation,
            "ghi_wm2": int(row["GHI_w/m2"]),
        },
        "hotspots": hotspot_dict.get(date, []),
    }

    # insert into mongo
    collection.insert_one(climate_doc)

In [ ]:
# print collection to check
for doc in collection.find():
    pprint.pprint(doc)

##### Task 2.2


#### a. Find climate data on 12th December 2023.


In [ ]:
# get climate data for 12/12/2023
query = datetime(2023, 12, 12).strftime("%Y-%m-%dT%H:%M:%S")

# find the document
result = collection.find_one({"date": query})

# print the document
pprint.pprint(result)

#### b. Find the latitude, longitude, surface temperature (°C), and confidence when the surface temperature (°C) was between 65 °C and 100 °C.


In [ ]:
# get surface temperature between 65 and 100
query = {"hotspots.surface_temperature_celcius": {"$gte": 65, "$lte": 100}}

# projection to only show hotspots
projection = {"_id": 0, "hotspots": 1}

# find the document
result = collection.find(query, projection)
for doc in result:
    for hotspot in doc["hotspots"]:
        pprint.pprint(
            {
                "latitude": hotspot["latitude"],
                "longitude": hotspot["longitude"],
                "surface_temperature_celcius": hotspot["surface_temperature_celcius"],
                "confidence": hotspot["confidence"],
            }
        )

#### c. Find date, surface temperature (°C), air temperature (°C), relative humidity and max wind speed on 15th and 16th of December 2023.


In [ ]:
# get climate data for 15/12/2023 and 16/12/2023
query = {
    "date": {
        "$in": [
            datetime(2023, 12, 15).strftime("%Y-%m-%dT%H:%M:%S"),
            datetime(2023, 12, 16).strftime("%Y-%m-%dT%H:%M:%S"),
        ]
    }
}

# projection to only show climate data
projection = {
    "_id": 0,
    "date": 1,
    "hotspots.surface_temperature_celcius": 1,
    "climate.air_temperature_celcius": 1,
    "climate.relative_humidity": 1,
    "climate.max_wind_speed": 1,
}

result = collection.find(query, projection)
for doc in result:
    pprint.pprint(doc)

#### d. Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.


In [ ]:
# get confidence between 80 and 100
query = {"hotspots.confidence": {"$gte": 80, "$lte": 100}}

# projection to only datetime, air temperature, surface temperature, and confidence
projection = {
    "_id": 0,
    "hotspots.datetime": 1,
    "climate.air_temperature_celcius": 1,
    "hotspots.surface_temperature_celcius": 1,
    "hotspots.confidence": 1,
}

# find the document
result_d = collection.find(query, projection)
for doc in result_d:
    pprint.pprint(doc)

#### e. Find the top 10 records with the highest surface temperature (°C).


In [ ]:
# get top 10 hotspots with highest surface temperature
query = [
    {"$unwind": "$hotspots"},
    {"$sort": {"hotspots.surface_temperature_celcius": -1}},
    {"$limit": 10},
    {"$project": {"_id": 0, "date": 1, "hotspots": 1}},
]

result = collection.aggregate(query)

for doc in result:
    pprint.pprint(doc)

#### f. Find the number of fires each day. You are required to only display the total number of fires and the date in the output.


In [ ]:
# get number of fires for each date
query = [{"$project": {"_id": 0, "date": 1, "number_of_fires": {"$size": "$hotspots"}}}]

result = collection.aggregate(query)

for doc in result:
    pprint.pprint(doc)

### g. Find the records of fires where the confidence is below 70.


In [ ]:
# get average surface temperature for each date
query = {"hotspots.confidence": {"$lte": 70}}

projection = {
    "_id": 0,
    "date": 1,
    "hotspots": {"$elemMatch": {"confidence": {"$lte": 70}}},
}

result = collection.find(query, projection)

for doc in result:
    pprint.pprint(doc)

#### h. Find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) and the date in the output.


In [ ]:
# get average surface temperature for each date
query = [
    {"$unwind": "$hotspots"},
    {
        "$group": {
            "_id": "$date",
            "average_surface_temperature": {
                "$avg": "$hotspots.surface_temperature_celcius"
            },
        }
    },
    {"$project": {"_id": 0, "date": "$_id", "average_surface_temperature": 1}},
]

result = collection.aggregate(query)

for doc in result:
    pprint.pprint(doc)

#### i. Find the top 10 records with the lowest GHI.


In [ ]:
# get average surface temperature for each date
query = collection.find().sort("climate.ghi_wm2", 1).limit(10)

for result in query:
    pprint.pprint(result)

#### j. Find the records with a 24-hour precipitation recorded between 0.20 to 0.35.


In [ ]:
# get average surface temperature for each date
query = {"climate.precipitation": {"$gte": "0.20G", "$lte": "0.35G"}}

projection = {"_id": 0, "date": 1, "climate.precipitation": 1}

result = collection.find(query)

for doc in result:
    pprint.pprint(doc)

In [ ]:
# latest date for streaming
query = collection.find({}, {"_id": 0, "date": 1}).sort("date", -1).limit(1)

for result in query:
    pprint.pprint(result)

##### Task 2.3


Assuming a high data ingestation rate, I recommend creating a compound index on `date` and `hotspots.surface_temperature_celcius` for optimisation. Indexes are used to speed up operations, particularly the retrieval of operations in a MongoDB collection. This compound index combines the highly selective/unique `date` field with the high-cardinality and non-repetitive `hotspots.surface_temperature_celcius` field, making it an ideal choice for the StopFire campaign's fire prediction and analysis needs.

Queries that involve both the `date` and `hotspots.surface_temperature_celcius` fields will benefit greatly from this index. The database can quickly narrow down the search results based on the specific date and surface temperature range, reducing the number of documents scanned and improving query performance. Furthermore, it could also be used in the future to enable effective fire prediction, analysis, and data modelling.


In [ ]:
from pymongo import ASCENDING, DESCENDING

# drop all indexes
collection.drop_indexes()

# create the compound index on `date` and `hotspots.surface_temperature_celcius`
collection.create_index(
    [("date", ASCENDING), ("hotspots.surface_temperature_celcius", ASCENDING)]
)

### References


https://www.mongodb.com/blog/post/performance-best-practices-indexing
